In [ ]:
import open3d as o3d
import torch
import numpy as np
import objaverse
import multiprocessing
from PIL import Image
import io
from matplotlib import pyplot as plt
from os import path
import os
import shutil
from tqdm.notebook import tqdm

# transforming object

In [ ]:
#uid = 'febdbce61b45472da3de7d9a9878ef7a'

# Nicobar Pigeon  40 cm
uid1 = "a995d6f5ea674aa7a7365d5372176b95"
# Eastern Rosella 30 cm
uid2 = "f3c9b82285c24ad6b8f683d285c9cd6c"

model_annot = objaverse.load_annotations([uid1,uid2])

processes = multiprocessing.cpu_count()
objects = objaverse.load_objects(uids=[uid1,uid2],download_processes=processes)
# from api
model_1 = o3d.io.read_triangle_mesh(objects[uid1], True)
model_2 = o3d.io.read_triangle_mesh(objects[uid2], True)
#model_3 = o3d.io.read_triangle_mesh(objects[uid3], True)


# from local
#model_o3d = o3d.io.read_triangle_mesh("/mnt/project/AT3DCV_Data/source_models/deathwing.glb", True)
model_bounding_box1 = model_1.get_axis_aligned_bounding_box()
model_bounding_box2 = model_2.get_axis_aligned_bounding_box()
#model_bounding_box3 = model_3.get_axis_aligned_bounding_box()

# scene related
scene = 'scene0024_00'
dataset_path = '/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_3d'
scene_path = f'train/{scene}_vh_clean_2.pth'
_path = path.join(dataset_path, scene_path)    

# load 3D scene
scene_array, scene_colors, scene_annot = torch.load(_path)

scene_o3d = o3d.geometry.PointCloud()
scene_o3d.points = o3d.utility.Vector3dVector(scene_array)
scene_o3d.colors = o3d.utility.Vector3dVector(scene_colors)

scene_bounding_box = scene_o3d.get_oriented_bounding_box()

In [ ]:
# transform the new object
# scale_factor = 0.2 / -model_bounding_box.max_bound[2]

# for birds
scale_factor = 2
model_1.scale(scale_factor+1, model_1.get_center())
model_2.scale(scale_factor+2, model_2.get_center())
#model_3.scale(scale_factor+5, model_3.get_center())
model_bounding_box1.scale(scale_factor, np.zeros(3))
model_bounding_box2.scale(scale_factor, np.zeros(3))
#model_bounding_box3.scale(scale_factor, np.zeros(3))


#for single objects (old code)
#model_o3d.scale(scale_factor, model_o3d.get_center())
#model_bounding_box.scale(scale_factor, np.zeros(3))

#translate birds
model_1.translate((4.795, 4.284, 1.12),relative=False)
#model_1.translate((2.570, 1.894, 0.23),relative=False)
#model_1.translate((3.290, 1.264, 0.56),relative=False)
#model_2.translate((1.253, 6.613, 0.5),relative=False)
model_2.translate((2.253, 5.613, 0.2),relative=False)
#model_2.translate((4.323, 5.391, 0.84),relative=False)
#model_3.translate((2.279,4.640,0.9),relative=False)

#rotate birds
model_1.rotate(model_1.get_rotation_matrix_from_xyz((np.pi /2, np.pi/1.5, 0 )), center=model_1.get_center())
model_2.rotate(model_2.get_rotation_matrix_from_xyz((np.pi /2, np.pi/4, 0 )), center=model_2.get_center())
#model_3.rotate(model_3.get_rotation_matrix_from_xyz((np.pi /2, np.pi/8, 0 )), center=model_3.get_center())

#model_o3d.translate(scene_bounding_box.center,relative=False)
#model_o3d.translate((0,-1,0))

#model_o3d.rotate(model_o3d.get_rotation_matrix_from_xyz((np.pi /2, np.pi/8, 0 )), center=model_o3d.get_center())

In [ ]:
o3d.visualization.draw_plotly([scene_o3d])

In [ ]:
geometries = [model_1,model_2,scene_o3d]
#geometries = [augmented_scene_o3d]
viewer = o3d.visualization.Visualizer()
viewer.create_window()
for geometry in geometries:
    viewer.add_geometry(geometry)
opt = viewer.get_render_option()
opt.show_coordinate_frame = True
opt.background_color = np.asarray([0.5, 0.5, 0.5])
viewer.run()
viewer.destroy_window()

# exporting pc

In [ ]:
def sample_points(model, amount, voxel_size):
    # sampling points from mesh
    down_model = model.sample_points_uniformly(amount)
    down_model = down_model.voxel_down_sample(voxel_size = voxel_size)

    if np.asarray(down_model.colors).size == 0:
        down_model.paint_uniform_color(np.array([0, 0, 0]))
    return down_model

def create_labels(model, label):
    model_labels = np.full(np.asarray(model.points).shape[0], label)
    print(f'labels created for total of {np.asarray(model.points).shape[0]} points')
    return model_labels

# if crop is necessary somehow
def crop_point_cloud_by_z(point_cloud, z_threshold):
    # Extract Z coordinates from point cloud
    points = np.asarray(point_cloud.points)
    z_values = points[:, 2]

    # Find points with Z coordinates lower than the threshold
    indices = np.where(z_values >= z_threshold)[0]

    # Create a new point cloud with the cropped points
    cropped_point_cloud = point_cloud.select_by_index(indices.tolist())

    return cropped_point_cloud

d_model_1 = sample_points(model_1, 3000, 0.015)
d_model_2 = sample_points(model_2, 3000, 0.015)
#d_model_3 = sample_points(model_3, 3000, 0.02)

labels_1 = create_labels(d_model_1, 20.0)
labels_2 = create_labels(d_model_2, 21.0)
#labels_3 = create_labels(d_model_3, 22.0)

# concatenating scene and object arrays
augmented_scene_points = np.concatenate([np.asarray(scene_o3d.points), np.asarray(d_model_1.points), np.asarray(d_model_2.points)])
augmented_scene_colors = np.concatenate([np.asarray(scene_o3d.colors), np.asarray(d_model_1.colors), np.asarray(d_model_2.colors)])
augmented_scene_labels = np.concatenate([scene_annot, labels_1, labels_2])

# saving as a torch file
OUT_DIR_3D = "/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/augmented/birds_new/scannet_3d"
TAG = "example"

torch.save((augmented_scene_points, augmented_scene_colors, augmented_scene_labels),
           path.join(OUT_DIR_3D, TAG, scene_path.split('/')[-1]))


In [ ]:
# just to visualize it in here
augmented_scene_o3d = o3d.geometry.PointCloud()
augmented_scene_o3d.points = o3d.utility.Vector3dVector(augmented_scene_points)
augmented_scene_o3d.colors = o3d.utility.Vector3dVector(augmented_scene_colors)
o3d.visualization.draw_geometries([augmented_scene_o3d])

In [ ]:
# test if written file is correct
aug_test = torch.load(path.join(OUT_DIR_3D, TAG, scene_path.split('/')[-1]))

sample_points  = aug_test[0]
sample_colors = aug_test[1]

aug_test_pcd = o3d.geometry.PointCloud()
aug_test_pcd.points = o3d.utility.Vector3dVector(np.asarray(sample_points))
aug_test_pcd.colors = o3d.utility.Vector3dVector(np.asarray(sample_colors))

o3d.visualization.draw_geometries([aug_test_pcd])

# exporting .glb file

In [ ]:
# need to export transformed object as .obj o we can read it with read_triangle_model() for rendering
o3d.io.write_triangle_mesh("/mnt/project/AT3DCV_Data/exported/NicobarPigeon_mesh.obj", model_1)
o3d.io.write_triangle_mesh("/mnt/project/AT3DCV_Data/exported/EasternRosella_mesh.obj", model_2)
#o3d.io.write_triangle_mesh("/mnt/project/AT3DCV_Data/exported/MousecoloredTyrannulet_mesh.obj", model_3)

In [ ]:
# to check if exported object is correct
exported_mesh_1 = o3d.io.read_triangle_mesh("/mnt/project/AT3DCV_Data/exported/NicobarPigeon_mesh.obj", True)
exported_mesh_2 = o3d.io.read_triangle_mesh("/mnt/project/AT3DCV_Data/exported/EasternRosella_mesh.obj", True)
#exported_mesh_3 = o3d.io.read_triangle_mesh("/mnt/project/AT3DCV_Data/exported/MousecoloredTyrannulet_mesh.obj", True)

In [ ]:
exported_mesh_1.textures

In [ ]:
o3d.visualization.draw_geometries([exported_mesh_1,exported_mesh_2,scene_o3d])

# projecting back (test with a certain view)

In [ ]:
# http://www.open3d.org/docs/latest/python_example/visualization/index.html#render-to-image-py

scene = 'scene0024_00'
scene_img = '2440'

extrinsic_matrix = np.loadtxt(f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/{scene}/pose/{scene_img}.txt')
    
intrinsic_matrix = np.loadtxt(f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/intrinsics.txt')

img_path = f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/{scene}/color/{scene_img}.jpg'
depth_path = f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/{scene}/depth/{scene_img}.png'

scannet_img = np.asarray(o3d.io.read_image(img_path))

scannet_depth = np.asarray(o3d.io.read_image(depth_path))

img_height, img_width = scannet_depth.shape

In [ ]:
render = o3d.visualization.rendering.OffscreenRenderer(img_width, img_height)

In [ ]:
render.scene.set_background([1, 1, 1, 0])
render.scene.show_axes(True)

#white = o3d.visualization.rendering.MaterialRecord()
#white.base_color = [1.0, 1.0, 1.0, 1.0]
#white.shader = "defaultLit"

#material = o3d.visualization.rendering.MaterialRecord()
#texture = np.asarray(model_o3d.textures).copy()
#texture = o3d.geometry.Image(texture)
#material.albedo_img = texture

_object = o3d.io.read_triangle_model("/mnt/project/AT3DCV_Data/exported/EasternRosella_mesh.obj")

render.scene.add_model("model", _object)

vfov = 2 * np.arctan(img_height / (2 * intrinsic_matrix[1, 1]))
eye = extrinsic_matrix[:3, 3]
lookat = eye + (extrinsic_matrix[:3, :3] @ np.array([0, 0, 1]))
up = (extrinsic_matrix[:3, :3] @ np.array([0, -1, 0]))

render.setup_camera(vfov*180/np.pi, lookat, eye, up)

In [ ]:
img_model = np.asarray(render.render_to_image())
depth_model = np.asarray(render.render_to_depth_image(z_in_view_space=True)) * 1000

In [ ]:
aug_depth = np.asarray(scannet_depth).copy()
aug_img = np.asarray(scannet_img).copy()

In [ ]:
aug_depth_interp = aug_depth.copy()

# very simple (but not smart) interpolation of missing depth values for masking of 3d model
mask = (aug_depth==0)
aug_depth_interp[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), aug_depth[~mask])

#mask = np.all(np.array([aug_depth == 0, depth_model < np.inf]), axis=0)
#mask = np.any(np.array([aug_depth > depth_model, mask]), axis=0)

model_mask = aug_depth_interp > depth_model

aug_depth[model_mask] = depth_model[model_mask]
aug_img[model_mask] = img_model[model_mask]

In [ ]:
fig.savefig('augmented_2d.png', format='png')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(scannet_img)
ax[1].imshow(scannet_depth)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(aug_img)
ax[1].imshow(aug_depth)

# project back to all views

In [ ]:
transformed_object_path_1 = "/mnt/project/AT3DCV_Data/exported/NicobarPigeon_mesh.obj"
transformed_object_path_2 = "/mnt/project/AT3DCV_Data/exported/EasternRosella_mesh.obj"

OUT_DIR_2D = "/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/augmented/birds_new/scannet_2d"
scene = 'scene0024_00'

# read intrinsics 
intrinsic_matrix = np.loadtxt(f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/intrinsics.txt')
# initialize paths
aug_scene_folder = path.join(OUT_DIR_2D,scene)
aug_color_path = path.join(aug_scene_folder, "color")
aug_depth_path = path.join(aug_scene_folder, "depth")
aug_pose_path  = path.join(aug_scene_folder, "pose")

# create folders if not exists
if not path.exists(aug_scene_folder):
   os.makedirs(aug_scene_folder)
if not path.exists(aug_color_path):
   os.makedirs(aug_color_path)
if not path.exists(aug_depth_path):
   os.makedirs(aug_depth_path)
if not path.exists(aug_pose_path):
   os.makedirs(aug_pose_path)

# original 2D data path
original_path = f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/{scene}'

# copy pose files if files does not match
# ori_p_files = os.listdir(path.join(original_path,"pose"))
# tgt_p_files = os.listdir(aug_pose_path)
# if not tgt_p_files == ori_p_files: 
#   for fname in files: shutil.copy2(os.path.join(original_pose_path,fname), aug_pose_path)


In [ ]:
# list of images ids
image_ids = [_id.split(".")[0] for _id in os.listdir(path.join(original_path,"color"))]

In [ ]:
# read only one to extract the size of the image so we create renderer only once at the beginning
ex_img = f'/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/{scene}/depth/{image_ids[0]}.png'
_h, _w= np.asarray(o3d.io.read_image(ex_img)).shape

In [ ]:
# create renderer
render = o3d.visualization.rendering.OffscreenRenderer(_w, _h)
render.scene.set_background([1, 1, 1, 0])
render.scene.show_axes(True)
_object1 = o3d.io.read_triangle_model(transformed_object_path_1)
render.scene.add_model("object1", _object1)
_object2 = o3d.io.read_triangle_model(transformed_object_path_2)
render.scene.add_model("object2", _object2)

render.scene.scene.set_sun_light(
            [1, 1, 1],  # direction
            [1, 1, 1],  # color
            100000)  # intensity
render.scene.scene.enable_sun_light(True)

#_object3 = o3d.io.read_triangle_model(transformed_object_path_3)
#render.scene.add_model("object3", _object3)

# vertical fov
vfov = 2 * np.arctan(_h / (2 * intrinsic_matrix[1, 1]))

In [ ]:
# loop over each image
_compact = "/mnt/project/AT3DCV_Data/Preprocessed_OpenScene/data/scannet_2d/"
for _id in tqdm(image_ids):
    extrinsic_matrix = np.loadtxt(f'{_compact}{scene}/pose/{_id}.txt')
    _img = np.asarray(o3d.io.read_image(f'{_compact}{scene}/color/{_id}.jpg'))
    _depth = np.asarray(o3d.io.read_image(f'{_compact}{scene}/depth/{_id}.png'))
        
    #necessary calculations for each image
    eye = extrinsic_matrix[:3, 3]
    lookat = eye + (extrinsic_matrix[:3, :3] @ np.array([0, 0, 1]))
    up = (extrinsic_matrix[:3, :3] @ np.array([0, -1, 0]))
    
    # change camera position
    render.setup_camera(vfov*180/np.pi, lookat, eye, up)
    
    img_model = np.asarray(render.render_to_image())
    depth_model = np.asarray(render.render_to_depth_image(z_in_view_space=True)) * 1000
    
    aug_depth = np.asarray(_depth).copy()
    aug_img = np.asarray(_img).copy()

    aug_depth_interp = aug_depth.copy()

    # very simple (but not smart) interpolation of missing depth values for masking of 3d model
    mask = (aug_depth==0)
    aug_depth_interp[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), aug_depth[~mask])

    #mask = np.all(np.array([aug_depth == 0, depth_model < np.inf]), axis=0)
    #mask = np.any(np.array([aug_depth > depth_model, mask]), axis=0)

    model_mask = aug_depth_interp > depth_model

    aug_depth[model_mask] = depth_model[model_mask]
    aug_img[model_mask] = img_model[model_mask]
    
    o3d.io.write_image(path.join(aug_color_path, f'{_id}.jpg'), o3d.geometry.Image(aug_img))
    o3d.io.write_image(path.join(aug_depth_path,f'{_id}.png'), o3d.geometry.Image(aug_depth))

    tqdm.write(f'{_id} is written', end="\r")

In [ ]:
# legacy

#diamond firetail 12 cm
uid1 ="f30e9661f2de4c9eb14bfe1bd8bf15e0"
# Blue-faced Honeyeater  29.5 cm
uid2 = "f1cb8473b258433ab9bb61f8a96b4867"